In [2]:
import csv
import time
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import datetime
# import time

In [3]:
file = open("Feature_mapping.csv", "w", newline = '', encoding="utf8")
f = csv.writer(file)

# Feature columns + label for each Userid
feature1 = "Still"
feature2 = "Running"

f.writerow(["Userid", feature1+ " Morning", feature1 + " Afternoon", feature1 + " Evening", feature1 + " Night", feature2+ " Morning", feature2 + " Afternoon", feature2 + " Evening", feature2 + " Night", "ADHD", "ADHDScore","Survey Dates"])

156

In [4]:
df = pd.read_csv("Activity_clean.csv")
df = df.sort_values(by=['Userid','senseStartTimeMillis'],ascending=[True,True])

# get the list of ordered ids
ids = df["Userid"].unique()

In [5]:
# dictionary of ADHD labels for each user id
ADHD_dict = {}
baseline = pd.read_csv("sassema baseline 3.16.2023.csv")
baseline['ID'] = baseline['ID'].fillna(0)

for index in baseline.index:
    if index > 1:
        if baseline["ID"][index] != 0:
            if baseline["ID"][index] == "773":
                baseline["ID"][index] = "773684"
            if baseline["ADHDSx_Dx"][index] == "No":
                ADHD_dict[int(baseline["ID"][index])] = 0
            elif baseline["ADHDSx_Dx"][index] == "Yes":
                ADHD_dict[int(baseline["ID"][index])] = 1

In [6]:
def identifyTime(time,weekday):
    t = "23:59:59"
    night = datetime.datetime.strptime(t, "%H:%M:%S").time()
    t = "07:00:00"
    morning = datetime.datetime.strptime(t, "%H:%M:%S").time()
    t = "12:00:00"
    afternoon = datetime.datetime.strptime(t, "%H:%M:%S").time()
    t = "17:00:00"
    evening = datetime.datetime.strptime(t, "%H:%M:%S").time()
    time = time[1:-2]
    time_obj = datetime.datetime.strptime(time, "%H:%M:%S").time()

    if time_obj < morning:
        return "night"
    if time_obj >= morning and time_obj < afternoon:
        return "morning"
    if time_obj >= afternoon and time_obj < evening:
        return "afternoon"
    if time_obj >= evening and time_obj <= night:
        return "evening"

In [7]:
def determineWeekday(date):
    time = date
    time = time[2:-1]
    time = time.replace("T", " ")
    time = time.replace("-", "/")
    datetime_object = datetime.datetime.strptime(time, '%Y/%m/%d %H:%M:%S')
    dayOfTheWeek = datetime_object.weekday()

    if dayOfTheWeek >= 0 and dayOfTheWeek <= 4:
        weekday = 1
    else:
        weekday = 0
    
    return weekday, dayOfTheWeek
    

In [8]:
def CalculateFeature(de_date, index, threshold, first, last, end_of_period, type_activity):
    thr = 0 
    toAdd = 0 

    # check if the element before is "Act"
    if first == 0:
        previousAct = 0
    else:
        if de_date['Activity'][index-1] != type_activity:
            previousAct = 0
        else:
            diff = de_date['senseStartTimeMillis'][index] - de_date['senseStartTimeMillis'][index-1]
            if diff > threshold:
                previousAct = 0
            else:
                previousAct = 1

    
    # check if the element after is "Act"
    if last == len(de_date['Activity']) - 1:
        afterAct = 0
    else:
        if de_date['Activity'][index+1] != type_activity:
            afterAct = 0
        else:
            diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
            if diff > threshold:
                afterAct = 0
            else:
                afterAct = 1
    
    # if the following element represent a different activity
    if afterAct == 0 and last != len(de_date['Activity']) - 1:
        diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
        if diff <= threshold:
            toAdd = diff
        else:
            toAdd = diff
            thr = 1
    
    # if the following element represent the same activity
    if afterAct == 1:
        diff = de_date['senseStartTimeMillis'][index+1] - de_date['senseStartTimeMillis'][index]
        toAdd = diff
     
    # if it is the last element  in the dataframe
    if last == len(de_date['Activity']) - 1:
        time = de_date['Final Date'][index] 
        time = time[1:-1]
        time = time.replace("-", "/")
        time += end_of_period
        convert = datetime.datetime.strptime(time,'%Y/%m/%d %H:%M:%S')
        
        epoch_time = convert.timestamp()
        gmt_epoch_time = int(epoch_time - datetime.timezone.utc.utcoffset(convert).total_seconds())*1000 - 4*3600000
        diff = gmt_epoch_time - de_date['senseStartTimeMillis'][index]

        if diff <= threshold:
            toAdd = diff
        else:
            toAdd = diff
            thr = 1
        
    return toAdd, thr, previousAct, afterAct


In [9]:
Userid = [] 
StillMorning  = []
StillAfternoon = []
StillEvening = []
StillNight = []
RunningMorning  = []
RunningAfternoon = []
RunningEvening = []
RunningNight = []
ADHD = []
ADHDScore = []
Dates = []

In [10]:
#this function will take in 2 string describing part of day, and activity type
#it returns a set of threshold
#Set the running threshold to be 
#morning: 2 hour 
#afternoon: 2 hour 
#evening: 2 hour
#night: 0 hour 
#features in the order of morning, afternoon, evening, night
RunningThreshold = [2*3600000, 2*3600000, 2*3600000,0]
StillThreshold = [3600000,3600000,3*3600000,7*3600000]



#Running
# threshold_night = 0
# threshold_morning = 2*3600000
# threshold_afternoon = 2*3600000
# threshold_evening = 2*3600000

def thresholdSelection(when,activityType):
    partDay = -1
    if when == "morning":
        partDay = 0
    elif when == "afternoon":
        partDay = 1
    elif when == "evening":
        partDay = 2
    elif when == "night":
        partDay = 3

    if activityType == 'Running':
        return RunningThreshold[partDay]
    if activityType == 'Still':
        return StillThreshold[partDay]


In [11]:
#calculate the total feature time for a specific part of day
#it takes in part of day (morning/afternoon/etc), activity threshold from threshold Selection
#it will return totalMorningRunning and toAddRunningMorning
def partofDayFeature(timeAct,thr,previousAct, afterAct,thresholdList, totalWhenAct, toAddActWhen):
    # totalWhenAct = 0

    # If it is a completely new bundle
    if previousAct == 0:
        # add bundle time to total time
        totalWhenAct += toAddActWhen
        # reset bundle time
        toAddActWhen  = 0
        # print("222222toAddStillMorning: ",toAddActWhen)
        
    # if thr has not been reached add to existing bundle 
    if thr == 0:
        toAddActWhen += timeAct
        # print("3333333toAddStillMorning: ",toAddActWhen)
    # if thr has been reached 
    else:
        if toAddActWhen < thresholdList:
            if timeAct > thresholdList:
                toAddActWhen = thresholdList
            else:
                toAddActWhen += timeAct
            # print("44444444toAddStillMorning: ",toAddActWhen)
        else:
        # if the time in between activities > 3600 and total time to add is already > 3600 (add 10 more minutes)
            toAddActWhen += 600000

    return totalWhenAct, toAddActWhen

In [12]:

for id in ids:
    # get survey dates for the current id
    surveydates = df.loc[df["Userid"]==id, 'Survey Date'].tolist()
    uniquesurveydates = np.array(surveydates)
    uniquesurveydates = np.unique(uniquesurveydates)
    surveydates = uniquesurveydates.tolist()

    # get a dataframe of the data corresponding to current id
    de_id = df.loc[(df["Userid"])==int(id)]


    # calculate features for each survey date 
    for date in surveydates:
        # retrive dataframe that contains information for current survey date
        de_date = 0
        de_date = df.loc[(df["Userid"]==id) & (df["Survey Date"]==date)]
        # unique dates
        uniq = df.loc[(df["Userid"]==id) & (df["Survey Date"]==date), 'Final Date'].tolist()
        uniq = np.array(uniq)
        uniq = np.unique(uniq)
        uniq = uniq.tolist()
        
        for index in de_date.index:
            score = de_date["ADHDScore"][index]
            break

        # total time of being still for a survey date
        totalNightStill = 0
        totalMorningStill = 0
        totalAfternoonStill = 0
        totalEveningStill = 0
        totalNightRunning = 0
        totalMorningRunning = 0
        totalAfternoonRunning = 0
        totalEveningRunning = 0
        
        
        toAddRunningNight = 0
        toAddRunningMorning = 0
        toAddRunningAfternoon = 0
        toAddRunningEvening = 0

        toAddStillNight = 0
        toAddStillMorning = 0
        toAddStillAfternoon = 0
        toAddStillEvening = 0
        
        # keep track of first and last
        first = 0
        last = 0
        
        # iterate through dataframe and calculate features        
        for index in de_date.index:
            # determine weekend or weekday
            weekday, dayOfTheWeek = determineWeekday(de_date["senseStartTime"][index])
            
            # determine part of the day 
            partOfDay = identifyTime(de_date["Time"][index],weekday)
            
            # toAddActWhen = 0

            if de_date["Activity"][index] == " 'Still'":
                if (partOfDay == "morning"):
                    # print("id---", id)
                    thre = thresholdSelection(partOfDay,"Still")  #selecting corresponding threshold value


                    timeStill, thr, previousStill, afterStill = CalculateFeature(de_date, index, thre, first, last, " 12:00:00", " 'Still'")
                    
                    # print("passed calculate feature")
                    # print("111111toAddStillMorning: ",toAddStillMorning)


                    totalMorningStill, toAddStillMorning = partofDayFeature(timeStill,thr, previousStill,afterStill, thre,totalMorningStill, toAddStillMorning)


                    if last == len(de_date["Activity"]) - 1:
                        totalMorningStill += toAddStillMorning

                
                    # print("totalMorningStill: ", totalMorningStill, "toAddStillMorning: ",toAddStillMorning)  
                
                if (partOfDay == "afternoon"):
                    thre = thresholdSelection(partOfDay,"Still")  #selecting corresponding threshold value


                    timeStill, thr, previousStill, afterStill = CalculateFeature(de_date, index, thre, first, last, " 17:00:00", " 'Still'")
    
                    totalAfternoonStill, toAddStillAfternoon = partofDayFeature(timeStill,thr, previousStill,afterStill, thre, totalAfternoonStill,toAddStillAfternoon)
                    
                    if last == len(de_date["Activity"]) - 1:
                        totalAfternoonStill += toAddStillAfternoon

                if (partOfDay == "evening"):
                    thre = thresholdSelection(partOfDay,"Still")  #selecting corresponding threshold value

                    timeStill, thr, previousStill, afterStill = CalculateFeature(de_date, index, thre, first, last, " 23:59:59", " 'Still'")
    
                    totalEveningStill, toAddStillEvening = partofDayFeature(timeStill,thr, previousStill,afterStill, thre, totalEveningStill, toAddStillEvening)
                    
                    if last == len(de_date["Activity"]) - 1:
                        totalEveningStill += toAddStillEvening
                        

                if (partOfDay == "night"):
                    thre = thresholdSelection(partOfDay,"Still")  #selecting corresponding threshold value

                    
                    timeStill, thr, previousStill, afterStill = CalculateFeature(de_date, index, thre, first, last, " 07:00:00", " 'Still'")
    
                    totalNightStill, toAddStillNight = partofDayFeature(timeStill,thr, previousStill,afterStill, thre, totalNightStill,toAddStillNight)
                    
                    if last == len(de_date["Activity"]) - 1:
                        totalNightStill += toAddStillNight

            
            if de_date["Activity"][index] == " 'Running'":
                if (partOfDay == "morning"):
                    # print("id---", id)
                    thre = thresholdSelection(partOfDay,"Running")  #selecting corresponding threshold value


                    timeRunning, thr, previousRunning, afterRunning = CalculateFeature(de_date, index, thre, first, last, " 12:00:00", " 'Running'")
                    
                    # print("passed calculate feature")
                    # print("111111toAddRunningMorning: ",toAddRunningMorning)


                    totalMorningRunning, toAddRunningMorning = partofDayFeature(timeRunning,thr, previousRunning,afterRunning, thre,totalMorningRunning, toAddRunningMorning)


                    if last == len(de_date["Activity"]) - 1:
                        totalMorningRunning += toAddRunningMorning

                
                    # print("totalMorningRunning: ", totalMorningRunning, "toAddRunningMorning: ",toAddRunningMorning)  
                
                if (partOfDay == "afternoon"):
                    thre = thresholdSelection(partOfDay,"Running")  #selecting corresponding threshold value


                    timeRunning, thr, previousRunning, afterRunning = CalculateFeature(de_date, index, thre, first, last, " 17:00:00", " 'Running'")
    
                    totalAfternoonRunning, toAddRunningAfternoon = partofDayFeature(timeRunning,thr, previousRunning,afterRunning, thre, totalAfternoonRunning,toAddRunningAfternoon)
                    
                    if last == len(de_date["Activity"]) - 1:
                        totalAfternoonRunning += toAddRunningAfternoon

                if (partOfDay == "evening"):
                    thre = thresholdSelection(partOfDay,"Running")  #selecting corresponding threshold value

                    timeRunning, thr, previousRunning, afterRunning = CalculateFeature(de_date, index, thre, first, last, " 23:59:59", " 'Running'")
    
                    totalEveningRunning, toAddRunningEvening = partofDayFeature(timeRunning,thr, previousRunning,afterRunning, thre, totalEveningRunning, toAddRunningEvening)
                    
                    if last == len(de_date["Activity"]) - 1:
                        totalEveningRunning += toAddRunningEvening
                        

                if (partOfDay == "night"):
                    thre = thresholdSelection(partOfDay,"Running")  #selecting corresponding threshold value

                    
                    timeRunning, thr, previousRunning, afterRunning = CalculateFeature(de_date, index, thre, first, last, " 07:00:00", " 'Running'")
    
                    totalNightRunning, toAddRunningNight = partofDayFeature(timeRunning,thr, previousRunning,afterRunning, thre, totalNightRunning,toAddRunningNight)
                    
                    if last == len(de_date["Activity"]) - 1:
                        totalNightRunning += toAddRunningNight

                    
            # keep track of first and last
            first = 1
            last += 1
       
        # calculate averages
        # calculate averages
        avgMorningStill = totalMorningStill / len(uniq)
        avgAfternoonStill = totalAfternoonStill / len(uniq)
        avgEveningStill = totalEveningStill / len(uniq)
        avgNightStill = totalNightStill / len(uniq)
        
        # append the averages to lists
        Userid.append(id) 
        ADHD.append(ADHD_dict[id])
        Dates.append(date)
        ADHDScore.append(score)


        StillMorning.append(avgMorningStill) 
        StillAfternoon.append(avgAfternoonStill) 
        StillEvening.append(avgEveningStill) 
        StillNight.append(avgNightStill)


        # calculate averages
        avgMorningRunning = totalMorningRunning / len(uniq)
        avgAfternoonRunning = totalAfternoonRunning / len(uniq)
        avgEveningRunning = totalEveningRunning / len(uniq)
        avgNightRunning = totalNightRunning / len(uniq)
        
        # append the averages to lists
        RunningMorning.append(avgMorningRunning) 
        RunningAfternoon.append(avgAfternoonRunning) 
        RunningEvening.append(avgEveningRunning) 
        RunningNight.append(avgNightRunning)

    
print(len(ADHD))

print(len(StillMorning))
Dates = [i.replace('"','') for i in Dates] #removing "" around the survey dates
print(Dates)
            


82
82
['2022-11-01', '2022-11-11', '2022-12-09', '2022-12-22', '2022-04-17', '2022-04-24', '2022-05-01', '2022-05-08', '2022-05-15', '2022-05-22', '2022-05-29', '2022-06-05', '2022-06-12', '2022-06-19', '2022-11-03', '2023-01-04', '2022-04-22', '2022-04-29', '2022-05-06', '2022-05-13', '2022-05-20', '2022-05-27', '2022-06-03', '2022-06-10', '2022-12-07', '2022-12-14', '2022-12-21', '2022-12-28', '2023-01-04', '2023-01-11', '2023-01-18', '2023-01-25', '2022-11-22', '2022-12-06', '2022-12-13', '2023-01-10', '2023-01-17', '2023-01-24', '2022-11-24', '2022-12-15', '2022-12-22', '2023-01-20', '2022-04-21', '2022-04-28', '2022-05-12', '2022-05-19', '2022-05-26', '2022-06-02', '2022-06-09', '2022-06-23', '2022-11-01', '2022-11-08', '2022-11-15', '2022-11-22', '2022-11-29', '2022-12-06', '2022-12-20', '2022-12-30', '2022-04-22', '2022-10-26', '2022-11-02', '2022-11-10', '2022-11-23', '2022-11-30', '2022-12-08', '2022-12-21', '2022-12-28', '2022-10-31', '2022-11-07', '2022-11-14', '2022-11-21',

In [13]:
def convertToMin(list):
    # returns a list of time in minutes 
    mins = []
    for milli in list:
        min = milli / 60000
        mins.append(min)
    return mins

In [14]:
#convert from milliseconds to minutes
StillMorning = convertToMin(StillMorning)
StillAfternoon = convertToMin(StillAfternoon)
StillEvening = convertToMin(StillEvening)
StillNight = convertToMin(StillNight)

RunningMorning = convertToMin(RunningMorning)
RunningAfternoon = convertToMin(RunningAfternoon)
RunningEvening = convertToMin(RunningEvening)
RunningNight = convertToMin(RunningNight)

In [15]:
#export to the file
print("USERID", (len(Userid)))
print("stillmorning", (len(StillMorning)))
print("ADHD", len(ADHD))
for l in range(len(Userid)):
    f.writerow([Userid[l], StillMorning[l], StillAfternoon[l], StillEvening[l], StillNight[l], RunningMorning[l], RunningAfternoon[l], RunningEvening[l],RunningNight[l],ADHD[l],ADHDScore[l],Dates[l]])

file.close()

USERID 82
stillmorning 82
ADHD 82
